<center><strong><font size=+3>Cross Continuous Wavelet Power Spectra</font></center>
<br><br>
</center>
<center><strong><font size=+2>Matyas Molnar and Bojan Nikolic</font><br></strong></center>
<br><center><strong><font size=+1>Astrophysics Group, Cavendish Laboratory, University of Cambridge</font></strong></center>

In [ ]:
import itertools
import os

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import AxesGrid, make_axes_locatable

import pywt
from wavelets.plotting import cws
from wavelets.utils import DATAPATH

In [ ]:
%matplotlib inline

In [ ]:
from matplotlib import rc
rc('font',**{'family':'serif','serif':['cm']})
rc('text', usetex=True)
rc('text.latex', preamble=r'\usepackage{amssymb} \usepackage{amsmath}')

In [ ]:
band_1 = [175, 334]
band_2 = [515, 694]

field_1 = [1.25, 2.70]
field_2 = [4.50, 6.50]
field_3 = [8.50, 10.75]

bad_ants = [0, 2, 11, 24, 50, 53, 54, 67, 69, 98, 122, 136, 139]

In [ ]:
save_fig_dir = '/lustre/aoc/projects/hera/mmolnar/figs'

In [ ]:
npz_b1 = np.load(os.path.join(DATAPATH, 'cwt_b1.npz'))
npz_b2 = np.load(os.path.join(DATAPATH, 'cwt_b2.npz'))
data1 = npz_b1['cwt']
data2 = npz_b2['cwt']

In [ ]:
lsts = npz_b1['lsts']
redg = npz_b1['redg']
chans1 = npz_b1['chans']
chans2 = npz_b2['chans']
freqs1 = npz_b1['freqs']
freqs2 = npz_b2['freqs']
wavelet = npz_b1['wavelet'].item()
scales = npz_b1['scales']
f_res = np.median(np.ediff1d(freqs1))

In [ ]:
f1 = np.where((lsts > field_1[0]) & (lsts < field_1[1]))[0]
f2 = np.where((lsts > field_2[0]) & (lsts < field_2[1]))[0]
f3 = np.where((lsts > field_3[0]) & (lsts < field_3[1]))[0]

In [ ]:
ew14 = np.where(redg[:, 0] == 2)[0]
ew28 = np.where(redg[:, 0] == 6)[0]

## CCWPS

### Jansky an delay units

#### Try Band 1 Field 1

In [ ]:
blg = ew28
f = f1
data = data1[..., f1, :]

In [ ]:
# cross-PS between all baseline pairs
bl_pairs = list(itertools.permutations(np.arange(blg.size), r=2))
bls1 = [i[0] for i in bl_pairs]
bls2 = [i[1] for i in bl_pairs]

In [ ]:
#we cross multiply adjacent time integrations, but all baseline pairs too
cwps_bl = data[...,::2, blg[bls1]] * np.conj(data[..., 1::2, blg[bls2]])

In [ ]:
#we incoherently average the power spectra over the redundant baseline axis and across the 
# remaining time bins in each field
cwps = np.nanmean(np.nanmean(cwps_bl, axis=3), axis=2)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5), dpi=125)

im1 = ax.pcolormesh(freqs1/1e6, scales, np.abs(cwps), norm=LogNorm(vmax=1e-1), cmap='jet')

divider = make_axes_locatable(ax)
cax1 = divider.append_axes('right', size='4%', pad=0.1)
plt.colorbar(im1, cax=cax1, extend='max')

ax.set_ylim((scales[-1]+0.5, scales[0]))
ax.set_yscale('log')
ax.set_title(r'$\mathrm{CCWPS}$')
ax.set_xlabel('Frequency [MHz]')
ax.set_ylabel('Delay [$\mu$s]')

# hack to get delay ylabel
ax.set_yticks([1/(f_res*1e-5), 1/(f_res*1e-6)])
ax.set_yticklabels([r'$10^1$', r'$10^0$'])
# for correct log
min_ticks = np.concatenate((1/(f_res*np.linspace(1e-5, 1e-6, 11)), \
                            1/(f_res*np.linspace(1e-6, 1e-7, 11))))[:16]
ax.set_yticks(min_ticks, minor=True)
ax.set_yticklabels([], minor=True)

# redshift?

f = lambda x: 1420.4/x - 1
g = lambda x: 1420.4/(1+x)

ax2 = ax.secondary_xaxis('top', functions=(f, g))
ax2.set_xlabel('Redshift')

fig.tight_layout()
plt.show()

#### All bands and fields

In [ ]:
blg = ew14
freqsb = [freqs1, freqs2]

bl_pairs = list(itertools.permutations(np.arange(blg.size), r=2))
bls1 = [i[0] for i in bl_pairs]
bls2 = [i[1] for i in bl_pairs]

fig = plt.figure(figsize=(7, 10), dpi=600)

grid = AxesGrid(fig, 111, nrows_ncols=(3, 2), axes_pad=0.15, cbar_location='bottom', \
                cbar_mode='single', cbar_size=0.15, cbar_pad=0.4, direction='column', aspect=False, \
                label_mode='L')
min_arr = []
c = 0
for d, data_ in enumerate([data1, data2]):
    
    freqs = freqsb[d]
    
    for f, field in enumerate([f1, f2, f3]):
        
        data = data_[..., field, :]
        cwps_bl = data[...,::2, blg[bls1]] * np.conj(data[..., 1::2, blg[bls2]])
        cwps = np.nanmean(np.nanmean(cwps_bl, axis=3), axis=2)
        
        min_arr.append(np.nanmin(np.abs(cwps)))

        im = grid[c].pcolormesh(freqs/1e6, scales, np.abs(cwps), norm=LogNorm(vmin=1e-5, vmax=1e-1), cmap='jet')
        
        c += 1
#         break
    
#     break

grid.cbar_axes[0].colorbar(im, extend='both', label=r'$|\mathrm{CWPS}|$')
    
for ax in grid:
    ax.set_ylim((scales[-1]+0.5, scales[0]))
    ax.set_yscale('log')

    # hack to get delay ylabel
    ax.set_yticks([1/(f_res*1e-5), 1/(f_res*1e-6)])
    ax.set_yticklabels([r'$10^1$', r'$10^0$'])
    # for correct log
    min_ticks = np.concatenate((1/(f_res*np.linspace(1e-5, 1e-6, 11)), \
                                1/(f_res*np.linspace(1e-6, 1e-7, 11))))[:19]
    ax.set_yticks(min_ticks, minor=True)
    ax.set_yticklabels([], minor=True)

for i in range(3):
    grid[i].set_ylabel(f'Field {i+1}\n\nDelay [$\mu$s]')
    
for i in (2, 5):
    grid[i].set_xlabel('Frequency [MHz]')
    
grid[0].set_title('Band 1')
grid[3].set_title('Band 2')

# grid[2].set_xticks([118, 120, 122, 124, 126, 128, 130, 132])
# grid[5].set_xticks([152, 154, 156, 158, 160, 162, 164, 166])

f = lambda x: (1420.4/x) - 1
g = lambda x: 1420.4/(1+x)

for i in (0, 3):
    ax2 = grid[i].secondary_xaxis('top', functions=(f, g))
    ax2.set_xlabel('\nRedshift')

# plt.savefig(os.path.join(save_fig_dir, 'cwps_14m_ew.pdf'), bbox_inches='tight')
plt.show()

### Cosmological units

Work in progress...

In [ ]:
import hera_pspec as hp
import pyuvdata
from hera_pspec.conversions import Cosmo_Conversions
from hera_pspec.data import DATA_PATH as HPDP

#### Cosmological units for power

In [ ]:
b = 0
freqs = freqsb[b]

In [ ]:
cosmo = hp.conversions.Cosmo_Conversions()
beamfile = os.path.join(HPDP, 'HERA_NF_pstokes_power.beamfits')
uvb = hp.pspecbeam.PSpecBeamUV(beamfile, cosmo=cosmo)

# find conversion factor from Jy to mK
Jy_to_mK = uvb.Jy_to_mK(freqs, pol='pI')

In [ ]:
uvb.compute_pspec_scalar(freqsb[0][0], freqsb[0][-1] + np.median(np.diff(freqsb[0])), 
                         len(freqsb[b]), pol='pI', little_h=True)

#### Convert delay to k_para

In [ ]:
cosmo.tau_to_kpara(z=7.9, little_h=True)*0.4/1e6